In [1]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# import tensorflow as tf

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

## Load the housing data using Pandas

In [2]:
 # Import our input dataset
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Clean the Data

In [3]:
# check data types
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [4]:
# convert date column to datetime
df['date']= pd.to_datetime(df['date'])

# check data types
df.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

In [5]:
# add separate columns for the sale year and month
df['yr_sold'] = df['date'].dt.year
df['month_sold'] = df['date'].dt.month

# drop the original date column
df.drop('date', axis=1, inplace=True)

# convert price to price in thousands
df['price(000s)'] = df['price']/1000

# drop the original price column
df.drop('price', axis=1, inplace=True)

# show updated data frame
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,month_sold,price(000s)
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,221.9
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,538.0
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,180.0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,604.0
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,510.0


In [6]:
# Encode the zipcode column using get_dummies
zipcode_dummies = pd.get_dummies(df['zipcode'])

# Concatenate the original dataframe with the zipcode_dummies dataframe
df = pd.concat([df, zipcode_dummies], axis=1)

# Drop the original zipcode column and drop the lat and long columns
df = df.drop(columns=["zipcode", "lat", "long"])

# display the updated dataframe
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,1,0,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,0
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [7]:
 # Encoding the yr_renovated column using a custom function
def encode_renovated(yr_renovated):
    """
    This function encodes renovation status by setting any year as 1 and 0 as 0.
    """
    if yr_renovated > 0:
        return 1
    else:
        return 0

# Call the encode_renovated function on the renovated column
df["renovated"] = df["yr_renovated"].apply(encode_renovated)


# Encoding the sqft_basement column using a custom function
def encode_basement(sqft_basement):
    """
    This function encodes basement status by setting any square footage as 1 and 0 as 0.
    """
    if sqft_basement > 0:
        return 1
    else:
        return 0

# Call the encode_basement function on the basement column
df["basement"] = df["sqft_basement"].apply(encode_basement)

# Review the DataFrame 
df.head()


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98155,98166,98168,98177,98178,98188,98198,98199,renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,1,0,0,0,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#count rows before dropping rows with duplicates or NA values
print(len(df))

# drop duplicate rows
df.drop_duplicates(subset="id", keep='first', inplace=True)

# drop rows containing NA values
df.dropna(axis=0, inplace=True)

#count rows after dropping rows with duplicates or NA values
print(len(df))

# show updated data frame
df.head()

21613
21436


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98155,98166,98168,98177,98178,98188,98198,98199,renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,1,0,0,0,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


### Identify and Remove Outliers

In [9]:

#identify outliers in price data using boxplots
#boxplot = df.boxplot(column="price(000s)", showcaps=True)

In [10]:
# Determine which data points are outlier (outside of the 1.5*IQR range)
prices = df['price(000s)']
quartiles = np.quantile(prices,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

# identify outliers
# potential_outliers = [price for price in prices if price < lower_bound or price > upper_bound]

# calculate percent of data to be dropped
# print(len(potential_outliers)/len(df)*100)

# drop rows with price value lower than lower_bound (outliers)
df = df[df['price(000s)'] >= lower_bound]

#length of updated data frame
print(len(df))

# drop rows with price value higher than upper_bound (outliers)
df = df[df['price(000s)'] <= upper_bound]

# length of updated data frame
print(len(df))

# show updated df
df.head()

21436
20297


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98155,98166,98168,98177,98178,98188,98198,98199,renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,1,0,0,0,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Determine which data points are outlier (outside of the 1.5*IQR range)
bedrooms = df['bedrooms']
quartiles = np.quantile(prices,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

# identify outliers
# potential_outliers = [price for price in prices if price < lower_bound or price > upper_bound]

# calculate percent of data to be dropped
# print(len(potential_outliers)/len(df)*100)

# drop rows with number of bedrooms lower than lower_bound (outliers)
df = df[df['bedrooms'] >= lower_bound]

#length of updated data frame
print(len(df))

# drop rows with number of bedrooms higher than upper_bound (outliers)
df = df[df['bedrooms'] <= upper_bound]

# length of updated data frame
print(len(df))

# show updated df
df.head()


20297
20297


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98155,98166,98168,98177,98178,98188,98198,98199,renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,1,0,0,0,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# # plot price of all houses
# x_values = df['id']
# y_values = df['price(000s)']
# plt.scatter(x_values, y_values)


# # identify outliers in subsets using boxplots
# df.boxplot(by='bedrooms', column=["price(000s)"], grid=False)
# df.boxplot(by='bathrooms', column=["price(000s)"], grid=False)
# df.boxplot(by='floors', column=["price(000s)"], grid=False)
# df.boxplot(by='waterfront', column=["price(000s)"], grid=False)
# df.boxplot(by='view', column=["price(000s)"], grid=False)
# df.boxplot(by='condition', column=["price(000s)"], grid=False)
# df.boxplot(by='grade', column=["price(000s)"], grid=False)
# df.boxplot(by='yr_built', column=["price(000s)"], grid=False)
# df.boxplot(by='zipcode', column=["price(000s)"], grid=False)
# df.boxplot(by='yr_sold', column=["price(000s)"], grid=False)

In [13]:
# calculate approximate bin size with "n" number of bins
# n=10
# df["price(000s)"].value_counts(bins=n, sort=False)

# Bin the price(000s) column using cut with $100k intervals
alt_price_labels = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]
price_bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]
# price_labels = ["0 to 100k",
#    "100k to 200k",
#    "200k to 300k",
#    "300k to 400k",
#    "400k to 500k",
#    "500k to 600k",
#    "600k to 700k",
#    "700k to 800k",
#    "800k to 900k",
#    "900k to 1000k",
#    "1000k to 1100k",
#    "1100k to 1200k"]
df["price_bins"] = pd.cut(df["price(000s)"], bins=price_bins, labels=alt_price_labels)

# Display the updated dataframe
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98166,98168,98177,98178,98188,98198,98199,renovated,basement,price_bins
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,1,0,0,0,0,0,300
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,1,1,600
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,200
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,1,700
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,600


In [14]:
 # Scaling the numeric columns
data_scaled = StandardScaler().fit_transform(df[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
       'floors', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated',
       #'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'yr_sold', 'month_sold']])

# Review the scaled data
data_scaled

array([[-0.3621819 , -1.48174323, -1.02935311, ..., -0.25616169,
        -0.68366252,  1.09469788],
       [-0.3621819 ,  0.27540177,  0.76310217, ..., -0.1814131 ,
        -0.68366252,  1.73776002],
       [-1.45888577, -1.48174323, -1.55806294, ..., -0.16551634,
         1.46270998, -1.47755066],
       ...,
       [-1.45888577, -1.83317223, -1.2356789 , ..., -0.39306925,
        -0.68366252, -0.19142639],
       [-0.3621819 ,  0.62683077, -0.48774792, ..., -0.42012757,
         1.46270998, -1.79908173],
       [-1.45888577, -1.83317223, -1.2356789 , ..., -0.4174969 ,
        -0.68366252,  1.09469788]])

In [15]:
# Create a DataFrame of the scaled data
data_scaled_df = pd.DataFrame(data_scaled, columns=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
                                                'floors', 'view', 'condition', 'grade', 'sqft_above',
                                                'sqft_basement', 'yr_built', 'yr_renovated',
                                                #'zipcode', 'lat', 'long',
                                                'sqft_living15', 'sqft_lot15', 'yr_sold', 'month_sold'])

# Replace the original data with the columns of information from the scaled Data
df["bedrooms"] = data_scaled_df["bedrooms"]
df["bathrooms"] = data_scaled_df["bathrooms"]
df["sqft_living"] = data_scaled_df["sqft_living"]
df["sqft_lot"] = data_scaled_df["sqft_lot"]
df["floors"] = data_scaled_df["floors"]
df["view"] = data_scaled_df["view"]
df["condition"] = data_scaled_df["condition"]
df["grade"] = data_scaled_df["grade"]
df["sqft_above"] = data_scaled_df["sqft_above"]
df["sqft_basement"] = data_scaled_df["sqft_basement"]
df["yr_built"] = data_scaled_df["yr_built"]
df["yr_renovated"] = data_scaled_df["yr_renovated"]
#df["zipcode"] = data_scaled_df["zipcode"]
#df["lat"] = data_scaled_df["lat"]
#df["long"] = data_scaled_df["long"]
df["sqft_living15"] = data_scaled_df["sqft_living15"]
df["sqft_lot15"] = data_scaled_df["sqft_lot15"]
df["yr_sold"] = data_scaled_df["yr_sold"]
df["month_sold"] = data_scaled_df["month_sold"]


# Review the DataFrame
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,98166,98168,98177,98178,98188,98198,98199,renovated,basement,price_bins
0,7129300520,-0.362182,-1.481743,-1.029353,-0.223532,-0.889079,0,-0.27027,-0.630048,-0.515778,...,0,0,0,1,0,0,0,0,0,300
1,6414100192,-0.362182,0.275402,0.763102,-0.183976,0.971487,0,-0.27027,-0.630048,-0.515778,...,0,0,0,0,0,0,0,1,1,600
2,5631500400,-1.458886,-1.481743,-1.558063,-0.115448,-0.889079,0,-0.27027,-0.630048,-1.478954,...,0,0,0,0,0,0,0,0,0,200
3,2487200875,0.734522,1.329689,-0.023515,-0.239683,-0.889079,0,-0.27027,2.462221,-0.515778,...,0,0,0,0,0,0,0,0,1,700
4,1954400510,-0.362182,-0.076027,-0.384585,-0.163154,-0.889079,0,-0.27027,-0.630048,0.447397,...,0,0,0,0,0,0,0,0,0,600


In [16]:
no_null_df = df.dropna()

# Data Splitting

In [17]:
 # Remove housing price target from features data
y = no_null_df["price_bins"].values
X = no_null_df.drop(columns=["price_bins","price(000s)"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=15, stratify=y)

# Model Training

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [19]:
# Create a linear regression model with scikit-learn
model = LinearRegression()
# Fit the model
model.fit(X_train, y_train)

LinearRegression()

In [20]:

# Make predictions
predictions = model.predict(X_test)

# Model evaluation
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print('Mean Squared Error : ', mse)
print('Root Mean Squared Error : ', rmse)
print('R-squared : ', r2)

Mean Squared Error :  22756.30412582796
Root Mean Squared Error :  150.8519278160805
R-squared :  0.48202202849753284


In [21]:
# Create a random forest classifier
rf_model = RandomForestRegressor()

# Fit the model
rf_model = rf_model.fit(X_train, y_train)

# Making predictions using the testing data
predictions = rf_model.predict(X_test)

In [22]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

ValueError: Shape of passed values is (689, 689), indices imply (2, 2)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix
Accuracy Score : 0.003984899328859061
Classification Report
              precision    recall  f1-score   support

       100.0       0.00      0.00      0.00         8
       200.0       0.00      0.00      0.00       199
       211.0       0.00      0.00      0.00         0
       212.0       0.00      0.00      0.00         0
       214.0       0.00      0.00      0.00         0
       230.0       0.00      0.00      0.00         0
       231.0       0.00      0.00      0.00         0
       234.0       0.00      0.00      0.00         0
       238.0       0.00      0.00      0.00         0
       240.0       0.00      0.00      0.00         0
       241.0       0.00      0.00      0.00         0
       243.0       0.00      0.00      0.00         0
       247.0       0.00      0.00      0.00         0
       248.0       0.00      0.00      0.00         0
       249.0       0.00      0.00      0.00         0
       250.0       0.00      0.00      0.00         0
    

# Model Evaluation